<a href="https://colab.research.google.com/github/Chaitanya-Atluri/Todo-list/blob/master/cnn_fin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
# -*- coding: utf-8 -*-
"""cnn-p1.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1EYig8tunIbtg2DXiP8IegiDr8oCCcJLG
"""

################################################################################
#
# LOGISTICS
#
#    Venkata sai krishna chaitanya atluri 
#    Vxa190013
#

# DESCRIPTION
#
#    MNIST image classification with an xNN written and trained in Python
#
# INSTRUCTIONS
#
#    1. Go to Google Colaboratory: https://colab.research.google.com/notebooks/welcome.ipynb
#    2. File - New Python 3 notebook
#    3. Cut and paste this file into the cell (feel free to divide into multiple cells)
#    4. Runtime - Run all
#
# NOTES
#
#    1. This does not use PyTorch, TensorFlow or any other xNN library
#
#    2. Include a short summary here in nn.py of what you did for the neural
#       network portion of code

        
#
#    3. Include a short summary here in cnn.py of what you did for the
#       convolutional neural network portion of code
#
#    4. Include a short summary here in extra.py of what you did for the extra
#       portion of code
#
################################################################################

################################################################################
#
# IMPORT
#
################################################################################

#
# you should not need any import beyond the below
# PyTorch, TensorFlow, ... is not allowed
#

import os.path
import urllib.request
import gzip
import math
import numpy             as np
import matplotlib.pyplot as plt
import time

################################################################################
#
# PARAMETERS
#
################################################################################

#
# add other hyper parameters here with some logical organization
#

# data
DATA_NUM_TRAIN         = 60000
DATA_NUM_TEST          = 10000
DATA_CHANNELS          = 1
DATA_ROWS              = 28
DATA_COLS              = 28
DATA_CLASSES           = 10
DATA_URL_TRAIN_DATA    = 'http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz'
DATA_URL_TRAIN_LABELS  = 'http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz'
DATA_URL_TEST_DATA     = 'http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz'
DATA_URL_TEST_LABELS   = 'http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz'
DATA_FILE_TRAIN_DATA   = 'train_data.gz'
DATA_FILE_TRAIN_LABELS = 'train_labels.gz'
DATA_FILE_TEST_DATA    = 'test_data.gz'
DATA_FILE_TEST_LABELS  = 'test_labels.gz'
EPOCHS                 = 5

# display
DISPLAY_ROWS   = 8
DISPLAY_COLS   = 4
DISPLAY_COL_IN = 10
DISPLAY_ROW_IN = 25
DISPLAY_NUM    = DISPLAY_ROWS*DISPLAY_COLS

################################################################################
#
# DATA
#
################################################################################

# download
if (os.path.exists(DATA_FILE_TRAIN_DATA)   == False):
    urllib.request.urlretrieve(DATA_URL_TRAIN_DATA,   DATA_FILE_TRAIN_DATA)
if (os.path.exists(DATA_FILE_TRAIN_LABELS) == False):
    urllib.request.urlretrieve(DATA_URL_TRAIN_LABELS, DATA_FILE_TRAIN_LABELS)
if (os.path.exists(DATA_FILE_TEST_DATA)    == False):
    urllib.request.urlretrieve(DATA_URL_TEST_DATA,    DATA_FILE_TEST_DATA)
if (os.path.exists(DATA_FILE_TEST_LABELS)  == False):
    urllib.request.urlretrieve(DATA_URL_TEST_LABELS,  DATA_FILE_TEST_LABELS)

# training data
# unzip the file, skip the header, read the rest into a buffer and format to NCHW
file_train_data   = gzip.open(DATA_FILE_TRAIN_DATA, 'r')
file_train_data.read(16)
buffer_train_data = file_train_data.read(DATA_NUM_TRAIN*DATA_ROWS*DATA_COLS)
train_data        = np.frombuffer(buffer_train_data, dtype=np.uint8).astype(np.float64)
train_data        = train_data.reshape(DATA_NUM_TRAIN, 1, DATA_ROWS, DATA_COLS)

# training labels
# unzip the file, skip the header, read the rest into a buffer and format to a vector
file_train_labels   = gzip.open(DATA_FILE_TRAIN_LABELS, 'r')
file_train_labels.read(8)
buffer_train_labels = file_train_labels.read(DATA_NUM_TRAIN)
train_labels        = np.frombuffer(buffer_train_labels, dtype=np.uint8).astype(np.int64)

# testing data
# unzip the file, skip the header, read the rest into a buffer and format to NCHW
file_test_data   = gzip.open(DATA_FILE_TEST_DATA, 'r')
file_test_data.read(16)
buffer_test_data = file_test_data.read(DATA_NUM_TEST*DATA_ROWS*DATA_COLS)
test_data        = np.frombuffer(buffer_test_data, dtype=np.uint8).astype(np.float64)
test_data        = test_data.reshape(DATA_NUM_TEST, 1, DATA_ROWS, DATA_COLS)

# testing labels
# unzip the file, skip the header, read the rest into a buffer and format to a vector
file_test_labels   = gzip.open(DATA_FILE_TEST_LABELS, 'r')
file_test_labels.read(8)
buffer_test_labels = file_test_labels.read(DATA_NUM_TEST)
test_labels        = np.frombuffer(buffer_test_labels, dtype=np.uint8).astype(np.int64)

np.random.seed(1)


# debug
# print(train_data.shape)   # (60000, 1, 28, 28)
# print(train_labels.shape) # (60000,)
# print(test_data.shape)    # (10000, 1, 28, 28)
# print(test_labels.shape)  # (10000,)

################################################################################
#
# YOUR CODE GOES HERE
#
################################################################################

#
# feel free to split this into some number of classes, functions, ... if it
# helps with code organization; for example, you may want to create a class for
# each of your layers that store parameters, performs initialization and
# includes forward and backward functions
#
# x = np.random.random((2,2,3))
# print(x)
#learning rate
lr= 0.001


#one hot vector for labels
one_hot_labels = np.zeros((train_labels.shape[0], 10))

for i in range(train_labels.shape[0]):
    one_hot_labels[i, train_labels[i]] = 1
one_hot_labels = np.array(one_hot_labels, dtype='float32')
# Relu activation function
def relu(a, derivative=False):
      if derivative:
        return np.greater(a,0).astype(int)
      else:
        return np.maximum(0,a)

def softmax(X):
      """ applies softmax to an input x"""
      exps = np.exp(X - np.max(X))
      return exps / np.sum(exps)


def cross_entropy(X,y):
    """
    X is the output from fully connected layer (num_examples x num_classes)
    y is labels (num_examples x 1)
    	Note that y is not one-hot encoded vector. 
    	It can be computed as y.argmax(axis=1) from one-hot encoded vectors of labels if required.
    """
    m = y.shape[0]
    p = softmax(X)
    # We use multidimensional array indexing to extract 
    # softmax probability of the correct label for each sample.
    # Refer to https://docs.scipy.org/doc/numpy/user/basics.indexing.html#indexing-multi-dimensional-arrays for understanding multidimensional array indexing.
    log_likelihood = -np.log(p[range(m),y])
    loss = np.sum(log_likelihood) / m
    return loss
  

def delta_cross_entropy(X,y):
    """
    X is the output from fully connected layer (num_examples x num_classes)
    y is labels (num_examples x 1)
    	Note that y is not one-hot encoded vector. 
    	It can be computed as y.argmax(axis=1) from one-hot encoded vectors of labels if required.
    """
    m = y.shape[0]
    grad = softmax(X)
    grad[range(m),y] -= 1
    grad = grad/m
    return grad


# normalize
def normalize(data):
    return(data/255).astype('float32')

train_data = normalize(train_data)
test_data = normalize(test_data)

# print(train_data[2])


# vectorize input array
train_data = train_data.reshape(train_data.shape[0],1,28*28)
test_data = test_data.reshape(test_data.shape[0],1,28*28)


#weight and bias for hidden layers
#sh being the shape anad num being number of neurons
def weight_init(data_temp,sh,num):
  w = np.random.randn(data_temp.shape[sh], num)*np.sqrt(1./data_temp.shape[sh])
  b = np.random.randn(num)
  return w,b;

# Hidden weight layer for input 
w1,b1=weight_init(train_data,2,1000)
# 2nd hidden layer
w2,b2=weight_init(w1,1,100)
# 3rd hidden layer
w3,b3=weight_init(w2,1,10)

def forward_weight_update(input,w,b,type):
  x=np.dot(input,w)
  z=np.add(x,b)
  if type == "hidden":
    a=relu(z)
  else:
    a=softmax(z)
  return x,z,a;

def backward_weight_update(w,b):
  dc_w = lr * w
  dc_b= lr * b.sum(axis=0)
  return dc_w,dc_b;

def softmax_derivative(out,k,a):
  dc_dz = out - one_hot_labels[k]
  dz_dw = a
  dc_w = np.dot(dz_dw.T, dc_dz)
  dc_b = dc_dz
  return dc_dz,dz_dw,dc_w,dc_b;

def hidden_derivative(w,dc_dz,z,a):
  dz_da = w
  dc_da = np.dot(dc_dz , dz_da.T)
  da_dz = relu(z, derivative=True)
  dz_dw = a
  dc_w = np.dot(dz_dw.T, da_dz * dc_da)
  dc_b = dc_da * da_dz
  return dz_da,dc_da,da_dz,dz_dw,dc_w,dc_b
print(w2[0])

# Feed forward network
for epoch in range(1):
  program_starts = time.time()
  loss=0
  for i,val in enumerate(train_data):
    temp_input = np.copy(val)
    # print(temp_input)
    #First layer
    x1,z1,a1 = forward_weight_update(temp_input, w1,b1,"hidden")
    #Second Layer
    x2,z2,a2 = forward_weight_update(a1, w2,b2,"hidden")

    #Third layer
    x3,z3,output = forward_weight_update(a2, w3,b3,"softmax")

    #backprop and derivatives
    #Final output layer dealing with softmax function
    dcost_dz3, dz3_dw3, dcost_w3, dcost_b3  = softmax_derivative(output,i,a2)
    # Second layer derivatives
    dz3_da2, dcost_da2, da2_dz2, dz2_dw2, dcost_w2, dcost_b2  =  hidden_derivative(w3,dcost_dz3,z2,a1)
    

     # First layer derivatives
    dcost_dz2 = dcost_da2*da2_dz2
    dz2_da1, dcost_da1, da1_dz1, dz1_dw1, dcost_w1, dcost_b1  =  hidden_derivative(w2,dcost_dz2,z1,temp_input)
    # Update weights
    # print(b3.shape, dcost_b3.shape)

    print(dcost_w2)
    w1, b1 = backward_weight_update(dcost_w1,dcost_b1)
    w2, b2 = backward_weight_update(dcost_w2,dcost_b2)   
    w3, b3 = backward_weight_update(dcost_w3,dcost_b3)  
    loss = np.sum(-one_hot_labels[i] * np.log(output))

    break
  now = time.time()
  print('Epoch: {} Loss: {:.6f} Time: {:.2f} secs'.format(epoch, loss, now-program_starts))
# error_cost.append(loss)
# output.shape
# print(w1)
# print(b1)

  # error_out = cross_entropy(output,train_labels[i])
    # print(error_out)
# temp_input

# cycle through the epochs

    # set the learning rate

    # cycle through the training data
        # forward pass
        # loss
        # back prop
        # weight update

    # cycle through the testing data
        # forward pass
        # accuracy
    # per epoch display (epoch, time, training loss, testing accuracy, ...)

# one_hot_labels[:10]
# train_data.shape[3]

import pickle
output = open('w1.pkl', 'wb')
pickle.dump(w1,output)
output = open('w2.pkl', 'wb')
pickle.dump(w2,output)
output = open('w3.pkl', 'wb')
pickle.dump(w3,output)
output = open('b1.pkl', 'wb')
pickle.dump(b1,output)
output = open('b2.pkl', 'wb')
pickle.dump(b2,output)
output = open('b3.pkl', 'wb')
pickle.dump(b3,output)

from google.colab import files
files.download('w2.pkl')
files.download('w3.pkl')
files.download('b1.pkl')
files.download('b2.pkl')
files.download('b3.pkl')

# print(w3)
################################################################################
#
# DISPLAY
#
################################################################################

#
# more code for you to write
#
test_count =0
for i, val in enumerate(test_data):
  x1 = np.dot(val, w1)
  z1 = np.add(x1,b1)
  a1 = relu(z1)
  # print(a1)

  #Second Layer
  x2 = np.dot(a1, w2)
  z2 = np.add(x2,b2)
  a2 = relu(z2)
  # print(a2)

  #Third layer
  x3 = np.dot(a2, w3)
  z3 = np.add(x3,b3)
  # print(z3)
  output = softmax(z3)
  # print(output, test_labels[i])
  if(np.argmax(output) == test_labels[i]):
    test_count +=1
print(test_count)

print(output.shape)

print(train_labels[10])

# accuracy display
# final value
# plot of accuracy vs epoch

# performance display
# total time
# per layer info (type, input size, output size, parameter size, MACs, ...)

# example display
# replace the xNN predicted label with the label predicted by the network




[-0.0492317  -0.01231534  0.03339495  0.01608108 -0.01102709  0.02242455
  0.01274421 -0.01754726 -0.03262975 -0.01438819 -0.05227837  0.04187349
 -0.01399111  0.02038802  0.00032959  0.00549054 -0.01984938  0.00323358
 -0.01785355 -0.07320418 -0.02649733  0.00342834  0.02703515 -0.08269768
 -0.03559248  0.02125909  0.02947881 -0.00933585  0.04123179  0.04307207
 -0.00234392  0.01786329  0.01015042  0.01053899 -0.03072504  0.00428253
  0.00014017  0.01396954 -0.03964925  0.02950051 -0.00150268  0.06345473
 -0.00480263 -0.01003738  0.05141312 -0.01748913  0.01604026  0.06969871
  0.01705649  0.02769872  0.00755494  0.02247241  0.02536635 -0.03547638
  0.06151644 -0.01344439 -0.03493614 -0.0156203   0.02239526 -0.01673837
 -0.00322556 -0.02819122 -0.02920526  0.01544511  0.05577555 -0.05152432
 -0.03975612  0.0110175  -0.05044548 -0.01289443 -0.03908151 -0.01317086
 -0.02120818  0.05647025 -0.02283039 -0.01958888  0.00932883 -0.03470356
 -0.00070394  0.06419917  0.02745562  0.08964704  0

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

958
(1, 10)
3
